In [1]:
import torch, random
import torchvision.transforms as transforms
from torchvision.io.video import read_video
from torchvision.models.video import r3d_18, R3D_18_Weights, R2Plus1D_18_Weights,r2plus1d_18
from torch.nn import functional as F
import cv2, os
from tqdm import tqdm
import numpy as np

#weights = R3D_18_Weights.DEFAULT
weights = R2Plus1D_18_Weights.DEFAULT
model = r2plus1d_18(weights=weights)
model.eval()

C:\ProgramData\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\ProgramData\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [2]:
transform = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.Resize((640, 480)),  
#     transforms.RandomCrop(224),     
    transforms.ToTensor(),          
#     transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),  # Normalize
])

In [3]:
X_tr = []
labels = []

In [4]:
model1 = torch.nn.Sequential(*list(model.children())[:-1])
model1.eval()

Sequential(
  (0): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

In [5]:
preprocess = weights.transforms()

#### Violance

In [6]:
path = "dataset/Violence/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2[:500]):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [23:51<00:00,  2.86s/it]


In [7]:
check = np.array(X_tr)
print(check.shape)

(500, 512)


#### NonViolance

In [8]:
%%time

path = "dataset/NonViolence/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2[:500]):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [22:44<00:00,  2.73s/it]

CPU times: total: 2h 26min 23s
Wall time: 22min 44s


In [9]:
ctr = np.array(X_tr)
print(ctr.shape)

(1000, 512)


In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np.array(labels)
labels = to_categorical(labels)
print(le.classes_, labels.shape)

['NonViolence' 'Violence'] (1000, 2)


#### Split the Video and Labels

In [11]:
print(ctr.shape,labels.shape)

(1000, 512) (1000, 2)


In [31]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=42, max_iter=256, hidden_layer_sizes=512, activation='logistic', 
                    solver='adam',learning_rate='adaptive',validation_fraction=0.10,shuffle=True,early_stopping=False,verbose=True)
clf.fit(ctr, labels)

Iteration 1, loss = 1.42860701
Iteration 2, loss = 1.19892989
Iteration 3, loss = 1.01477185
Iteration 4, loss = 0.84800305
Iteration 5, loss = 0.71019130
Iteration 6, loss = 0.61741170
Iteration 7, loss = 0.52989608
Iteration 8, loss = 0.46354341
Iteration 9, loss = 0.41752473
Iteration 10, loss = 0.38013999
Iteration 11, loss = 0.35381051
Iteration 12, loss = 0.33247051
Iteration 13, loss = 0.31157916
Iteration 14, loss = 0.29314003
Iteration 15, loss = 0.28125477
Iteration 16, loss = 0.26658930
Iteration 17, loss = 0.25690373
Iteration 18, loss = 0.24815700
Iteration 19, loss = 0.23830978
Iteration 20, loss = 0.23130259
Iteration 21, loss = 0.22073997
Iteration 22, loss = 0.22186353
Iteration 23, loss = 0.21499944
Iteration 24, loss = 0.20474478
Iteration 25, loss = 0.19593744
Iteration 26, loss = 0.19135187
Iteration 27, loss = 0.18757140
Iteration 28, loss = 0.18096997
Iteration 29, loss = 0.17534989
Iteration 30, loss = 0.17190284
Iteration 31, loss = 0.16598687
Iteration 32, los

MLPClassifier(activation='logistic', hidden_layer_sizes=512,
              learning_rate='adaptive', max_iter=256, random_state=42,
              verbose=True)

In [32]:
import pickle

pickle.dump(clf,open("model.pkl",'wb'))
pickle.dump(le,open("labels.pkl",'wb'))

### Test

In [33]:
import cv2
import pickle
import numpy as np

clf = pickle.load(open("model.pkl",'rb')) 
le = pickle.load(open("labels.pkl",'rb'))

In [34]:
le.classes_

array(['NonViolence', 'Violence'], dtype='<U11')

In [35]:
video = "test/violance4.mp4"

cap = cv2.VideoCapture(video)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = 512
frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
frame_numbers.sort()
frames = []
for frame_number in frame_numbers:    
    ret, frame = cap.read()        
    if frame is None:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(480,640))
    frames.append(frame)
input_ = np.array(frames)
video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
video_tensor = preprocess(video_tensor)
video_tensor = video_tensor.unsqueeze(0)
with torch.no_grad():
    my_output = model1(video_tensor)
pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
rgb_feature = pooled_output.flatten().cpu().numpy()
new_feature = np.expand_dims(rgb_feature,axis=0)
pred=clf.predict_proba(new_feature)[0]
print(le.inverse_transform([np.argmax(pred)])[0], pred[np.argmax(pred)], pred)            

NonViolence 0.90725386 [0.90725386 0.05856665]
